In [1]:
import csv
import json, codecs

import pandas as pd
from itertools import groupby
from tqdm import tqdm 

In [2]:
file_path = "SINGGALANGv2.tsv"
df_raw = pd.read_csv(file_path,
                     sep="\t",
                     names=["token", "ne"],
                     skip_blank_lines=False,
                     quoting=csv.QUOTE_NONE,
                     encoding="utf-8")
# df_raw = df_raw.loc[:39]
# df_raw = df_raw.loc[40:100]
# df_raw = df_raw.loc[19:65]
# df_raw = df_raw.loc[83:100]
# df_raw = df_raw.loc[:100]

In [3]:
list_sentence = []
list_entity = []

list_tmp = []

for row in df_raw.itertuples():
    
    if pd.isna(row.token) != True and pd.isna(row.ne) != True:
        list_tmp.append(tuple((row.token, row.ne)))
    else:
        list_sentence.append([e[0] for e in list_tmp])
        list_entity.append([e[1] for e in list_tmp])
        list_tmp = []

In [4]:
def sequences_chunking(list_token, list_ne):
    """
    Kelompokkan sekuens token dengan named entity yang bertetangga.
    Simpan sekuens yang merupakan named entity
    """
    list_tmp = []
    list_res = []
    curr_offset = 0
    for ix, ex in enumerate(list_ne):
        tkn = list_token[ix]
        if ix == 0:
            if ex != "O": list_tmp.append([curr_offset, tkn, ex])
            curr_offset += len(list_token[ix]) + 1
        elif ix < len(list_ne)-1:
            if ex == list_ne[ix-1]:
                if ex != "O": list_tmp.append([curr_offset, tkn, ex])
            else:
                list_res.append(list_tmp)
                list_tmp = []
                if ex != "O": list_tmp.append([curr_offset, tkn, ex])
            curr_offset += len(list_token[ix]) + 1
        else:
            if ex != "O": list_tmp.append([curr_offset, tkn, ex])
            list_res.append(list_tmp)
            list_tmp = []
    list_res = [i for i in list_res if i != []]
    return list_res

def entities_extraction(list_token, list_ne):
    """
    Ekstraksi informasi entitas.
    Simpan daftarnya untuk setiap kalimat
    """
    list_res = []
    for ex in sequences_chunking(list_token, list_ne):
        text = " ".join([ey[1] for ey in ex])
        label = [ey[2] for ey in ex][0]
        start_offset = [ey[0] for ey in ex][0]
        end_offset = start_offset + len(text)
        list_res.append([text, label, start_offset, end_offset])
    return list_res

list_singgalang = []
ne_id = 0
for ix, ex in tqdm(enumerate(list_sentence)):
    list_ne_info = entities_extraction(list_sentence[ix], list_entity[ix])
    list_ents = []
    for ey in list_ne_info:
        list_ents.append({
            "entity_id": ne_id,
            "text": ey[0],
            "label": ey[1],
            "start_offset": ey[2],
            "end_offset": ey[3]
        })
        ne_id+=1
    
    dict_data = {
        "doc_id": "Singgalang#" + str(ix),
        "doc_text": " ".join(ex),
        "entities": list_ents
    }
    list_singgalang.append(dict_data)

with open("singgalang.json", "wb") as f:
    json.dump(list_singgalang,
              codecs.getwriter("utf-8")(f),
              ensure_ascii=False,
              indent = 4)

48981it [00:00, 61302.93it/s]


---